In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Análisis exploratorio + Pre-procesamiento - Dataset HAM10000

In [ ]:
os.makedirs('./ISIC_HAM10000', exist_ok=True)

In [ ]:
data = pd.read_csv('./ISIC_HAM10000/HAM10000_metadata.csv')
data

In [ ]:
# Eliminamos columnas no necesarias para el análisis
columnas_a_eliminar = ['lesion_id', 'dx_type']  
data = data.drop(columns=columnas_a_eliminar)
data = data.dropna()

In [ ]:
for column in data.columns:
    unique_values = data[column].unique()
    print(f"Atributo: {column}, Valores únicos: {unique_values}")

In [ ]:
# Comprobación de valores null por columna
valores_nulos_por_columna = data.isnull().sum()
print(valores_nulos_por_columna)

In [ ]:
info_df = data.info()
print(info_df)

In [ ]:
# Remapeo de las variables del atributo 'diagnosis' para estandarizarlo con el resto de grupos
diagnosis_mapping = {
'nv': 'nevus',                      
'bcc': 'carcinoma',        
'mel': 'melanoma',                   
'bkl': 'keratosis',        
'akiec': 'carcinoma',           
'vasc': 'vascular lesion',             
'df': 'dermatofibroma',              
}  

data.rename(columns={'dx': 'diagnosis'}, inplace=True)
data['diagnosis'] = data['diagnosis'].replace(diagnosis_mapping)

In [ ]:
count_data = data['diagnosis'].value_counts().reset_index()
count_data.columns = ['diagnosis', 'Count']
ax = sns.barplot(data=count_data, x='Count', y='diagnosis', hue='diagnosis', dodge=False, legend=False)
plt.xlabel('Número de pacientes')
plt.ylabel('Diagnóstico')
plt.title('Pacientes según el tipo de diagnóstico')
plt.show()

In [ ]:
# Contamos los valores de 'age_approx', incluyendo NaN
pvc = data['age'].value_counts(dropna=False)
#edad_data = data.dropna(subset=['age'])

In [ ]:
age_min = data["age"].min()
age_max = data["age"].max()
age_mean = data["age"].mean()
age_std = data["age"].std()

print("Estadísticas de edad de los pacientes:")
print(f"Mínimo: {age_min}")
print(f"Máximo: {age_max}")
print(f"Media: {age_mean}")
print(f"Desviación estándar: {age_std}")

sns.set(style="whitegrid", palette="pastel")
sns.histplot(data=data, x='age', bins=30, kde=True, color='#6AB7F9')
plt.xlabel('Edad Aproximada')
plt.ylabel('Número de Pacientes')
plt.title('Distribución de Edades de los Pacientes')
plt.show()

In [ ]:
# Filtramos datos por género
female_px = data[data.sex == 'female']
male_px = data[data.sex == 'male']

# Configuramos estilo y paleta de colores de Seaborn
sns.set(style="whitegrid", palette="pastel")

# Creamos histogramas separados por género
sns.histplot(data=female_px, x='age', color='#FABCB7', label='mujer', kde=True)
sns.histplot(data=male_px, x='age', color='#B7D5FA', label='hombre', kde=True)
plt.xlabel('Edad Aproximada')
plt.ylabel('Número de Pacientes')
plt.title('Distribución de Edades de los Pacientes por Género')
plt.legend()
plt.show()

In [ ]:
data.localization = data.localization.replace('', np.nan)
data.localization.value_counts(dropna=False)

In [ ]:
# Definimos un diccionario de mapeo de valores específicos de 'Diagnosis' a 'benigno' o 'maligno'
mapeo_diagnosis = {
    'dermatofibroma': 'benign',
    'nevus': 'benign',
    'keratosis': 'benign',
    'vascular lesion': 'benign',
    'melanoma': 'malignant',
    'carcinoma': 'malignant',
}

# Aplicamos el mapeo a la columna 'Diagnosis' y asigna los resultados a una nueva columna llamada 'benign_malignant'
data['benign_malignant'] = data['diagnosis'].map(mapeo_diagnosis)

In [ ]:
data

In [ ]:
count_data = data['benign_malignant'].value_counts().reset_index()
count_data.columns = ['Diagnosis', 'Count']

# Count de casos benignos y malignos
num_benign = count_data[count_data['Diagnosis'] == 'benign']['Count'].values[0]
num_malignant = count_data[count_data['Diagnosis'] == 'malignant']['Count'].values[0]

ax = sns.barplot(data=count_data, x='Diagnosis', y='Count', hue='Diagnosis', dodge=False, legend=False)
plt.xlabel('Tipo')
plt.ylabel('Count')
plt.title('Ratio de benigno versus maligno\nBenignos: {} | Malignos: {}'.format(num_benign, num_malignant))
plt.show()

In [ ]:
# Eliminamos las muestras de diagnosis de tipo acral debido a que no se comparte con el resto de datasets y solo existen 7 muestras
data = data[data['localization'] != "acral"]

In [ ]:
# Filtramos filas donde la columna 'x' es igual a 'tipo a'
indices_a_eliminar = data[data['localization'] == 'unknown'].index

# Eliminamos las filas con los índices obtenidos
data = data.drop(indices_a_eliminar)

In [ ]:
# Mapeamos las localizaciones para que se adecuen al resto de datasets
category_mapping = {
    'scalp': 'head/neck',
    'ear': 'head/neck',
    'face': 'head/neck',
    'back': 'posterior torso',
    'trunk': 'anterior torso',
    'chest': 'anterior torso',
    'upper extremity': 'upper extremity',

    'abdomen': 'anterior torso',
    'lower extremity': 'lower extremity',
    'genital': 'oral/genital',
    'neck': 'head/neck',
    'hand': 'palms/soles',
    'foot': 'palms/soles',
}

data['localization'] = data['localization'].replace(category_mapping)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x='diagnosis', hue='localization', data=data, palette='pastel')
for i,v in enumerate(data.localization.unique()):
  try:
    ax.bar_label(ax.containers[i])
  except:
    continue

In [ ]:
# Filtramos el DataFrame para incluir solo las muestras de tipo "nevus"
muestras_nevus = data[data['diagnosis'] == 'nevus']

# Creamos un DataFrame vacío para almacenar las muestras después de la división
df_dividido = pd.DataFrame(columns=data.columns)

In [ ]:
muestras_nevus

In [ ]:
# Contamos el número de muestras por cada tipo de diagnosis
conteo_diagnosis = data['diagnosis'].value_counts()
print(conteo_diagnosis)

In [ ]:
#Eliminamos las muestras de tipo Nan para anatom_site_general y age
data = data.dropna(subset=['diagnosis'])
data = data.dropna(subset=['age'])
data = data.dropna(subset=['sex'])
data = data[data['sex'] != 'unknown']
data = data.dropna(subset=['localization'])

In [ ]:
# Categorizamos las variables para poder utilizarlas en el modelo 
sex_mapping = {'male': 0, 'female': 1}
benign_malignant_mapping = {'benign': 0, 'malignant': 1}

anatom_site_general_mapping = {
'head/neck': 0,          
'posterior torso': 1,    
'lower extremity': 2,  
'anterior torso': 3,   
'upper extremity': 4,    
'palms/soles': 5,         
'lateral torso': 6,        
'oral/genital': 7 }

data.rename(columns={'localization': 'anatom_site_general'}, inplace=True)

data['sex'] = data['sex'].replace(sex_mapping)

data.rename(columns={'image_id': 'id'}, inplace=True)
data.rename(columns={'age_approx': 'age'}, inplace=True)

data['age'] = data['age'].astype(int)
data['sex'] = data['sex'].astype(int) 

In [ ]:
data

In [ ]:
valores_unicos = data['benign_malignant'].unique()
valores_unicos

In [ ]:
# Ahora obtenemos una lista de los nombres de archivos correspondientes a las imágenes dermoscópicas
archivos_dermoscopic = data['id'].tolist()
archivos_dermoscopic = [id + ".jpg" for id in archivos_dermoscopic]

# Ruta de la carpeta que contiene todas las imágenes
carpeta = './ISIC_HAM10000/Dataset/'

# Obtenemos una lista de todos los archivos en la carpeta
archivos_totales = os.listdir(carpeta)

In [ ]:
import shutil

# Si el archivo no es imagen dermatoscopica y existe lo eliminamos de la carpeta de imágenes
for archivo in archivos_totales:
    if archivo not in archivos_dermoscopic:
        ruta_completa = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_completa):
            os.remove(ruta_completa)
    else:
        # Obtenemos la fila específica del dataset para el archivo actual
        nombre_sin_extension, extension = os.path.splitext(archivo)
        fila_especifica = data.loc[data['id'] == nombre_sin_extension]

        # Obtenemos la etiqueta de la fila
        etiqueta = fila_especifica['benign_malignant'].values[0]  # Asumiendo que 'benign_malignant' es el nombre de la columna
        if etiqueta == 'benign':
            shutil.copy(os.path.join(carpeta, archivo), "./ISIC_HAM10000/Dataset/Benigno/")
        elif etiqueta == 'malignant':
            shutil.copy(os.path.join(carpeta, archivo), "./ISIC_HAM10000/Dataset/Maligno/")

In [ ]:
data['benign_malignant'] = data['benign_malignant'].replace(benign_malignant_mapping)

In [ ]:
# Guardamos nuestro dataset procesado para el entrenamiento
data.to_csv('./ISIC_HAM10000/final_metadata_HAM10000.csv', index=False)